In [23]:
import gradio as gr
from transformers import TFAutoModelForSeq2SeqLM, AutoTokenizer
import tensorflow as tf
import re
import time

In [24]:
# mount google drive for saving results and models
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
# LOAD BEST MODEL
print("Loading Model....")
MODEL_PATH = '/content/drive/MyDrive/chatbot_model_Higher_LR_Extended'
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = TFAutoModelForSeq2SeqLM.from_pretrained(MODEL_PATH)
print("Model loaded successfully!!")

Loading Model....


/usr/local/lib/python3.12/dist-packages/tf_keras/src/initializers/initializers.py:121: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(
All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at /content/drive/MyDrive/chatbot_model_Higher_LR_Extended.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


Model loaded successfully!!


In [26]:
def is_in_domain(query):
    """Check if query is related to customer service"""
    domain_keywords = [
        'order', 'cancel', 'refund', 'payment', 'shipping', 'delivery',
        'account', 'invoice', 'subscription', 'track', 'return', 'purchase',
        'help', 'issue', 'problem', 'support','password'
    ]
    for keyword in domain_keywords:
        if keyword in query.lower():
            return True
    return False

In [27]:
 # function for taking in input and returning chatbot output
def display_output(user_input, history):

    # fall back message if message is not in domain
    if not is_in_domain(user_input):
        fallback = ("I apologize, but I can only assist with customer service queries "
                   "related to orders, refunds, shipping, accounts, and payments. "
                   "Please contact our support team for other inquiries at 1-800-123-4567.")
        partial = ""
        for word in fallback.split():
          partial += word + " "
          time.sleep(0.05)
          yield partial.strip()

        return

    query = f"Answer the customer service query:{user_input}" # format query to fit model input


    inputs = tokenizer(query, return_tensors='tf', padding=True, truncation=True) # tokenize inut
    outputs = model.generate(**inputs, max_new_tokens = 256, num_beams=1, do_sample=False) # generate output
    response = tokenizer.decode(outputs[0], skip_special_tokens=True) # decode output
    words = response.split()
    words = response.split()
    partial_response = ""
    for word in words:
        partial_response += word + " "
        time.sleep(0.05)  # Add 50ms delay between words (adjust as needed)
        yield partial_response.strip()




    # for i in range(len(response)):
    #     # time.sleep(0.1)
    #     yield response[: i+1]



In [ ]:
import gradio as gr
# gradio chatbot interface
gr.ChatInterface(
    fn=display_output,
    type="messages",
    title="💬 Customer Support Assistant",
    description=(
        "Welcome to the **Customer Service Chatbot** powered by Google FLAN-T5 and the Bitext dataset.\n\n"
        "You can ask questions related to customer support, such as:\n"
        "- Cancelling or tracking an order\n"
        "- Requesting a refund\n"
        "- Updating your account details and more FAQs\n\n"
        "💡 *Type your question below and press Enter or click 'Submit' to get a response.*"
    ),
    examples=[
        ["I want to cancel my order #12345"],
        ["Can you help me track my shipment?"],
        ["I'd like to request a refund for my last purchase."],
        ["How do I change my account password?"],
        ["What's your return policy?"],
        ["I want to change my shipping address for order 98765"],
    ],
    textbox=gr.Textbox(
        placeholder="Ask your question here... (e.g. 'How can I cancel my order?')",
        container=False,
        autoscroll=True,
        scale=7
    ),
    theme=gr.themes.Base(primary_hue="blue").set(
        body_background_fill="#f1f5f9",  # Light slate gray - easy on eyes
        body_background_fill_dark="#f1f5f9",
        body_text_color="#0f172a",       # Dark text for light background
        body_text_color_dark="#0f172a",
        block_label_text_color="#334155", # Slightly lighter for labels
        block_title_text_color="#1e293b",
        input_background_fill="#ffffff",  # White input fields
        button_primary_text_color="#ffffff",
        color_accent="#0284c7",  # Blue accent
        color_accent_soft="#0284c7",
    ),
).launch(share=True, debug=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://b1af794e83af85565a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
